In [13]:
import pandas as pd

# Load the gold recovery dataset
df_gold = pd.read_csv('../Dataset/gold_recovery_full.csv', parse_dates=['date'])

# Load the refined gold price dataset
df_price = pd.read_csv('../Dataset/Refined Gold Futures Historical Data.csv', parse_dates=['Date'])

# 2. Rename columns for consistency
df_price.rename(columns={'Date': 'date', 'Price': 'Close'}, inplace=True)

# 3. Convert both date columns to date-only (dropping the time component)
df_gold['date'] = df_gold['date'].dt.date
df_price['date'] = df_price['date'].dt.date

# 4. Reindex the price data to cover every day within the gold data's range and forward-fill missing prices
# Convert df_price 'date' back to datetime for reindexing
df_price['date'] = pd.to_datetime(df_price['date'])
df_price.set_index('date', inplace=True)

# Create a full date range from the min to max date in the gold dataset
full_date_range = pd.date_range(start=df_gold['date'].min(), end=df_gold['date'].max(), freq='D')

# Reindex the price DataFrame to include every day in the range
df_price = df_price.reindex(full_date_range)

# Forward-fill missing price values (i.e., weekends/holidays)
df_price = df_price.ffill()

# Reset the index so that 'date' becomes a column again, and convert it to date-only
df_price.reset_index(inplace=True)
df_price.rename(columns={'index': 'date'}, inplace=True)
df_price['date'] = df_price['date'].dt.date

# 5. Merge the two datasets on the 'date' column (left join to keep all gold recovery rows)
df_merged = pd.merge(df_gold, df_price[['date', 'Close']], on='date', how='left')

# Calculate profit: final.output.concentrate_au * gold price
df_merged['profit'] = df_merged['final.output.concentrate_au'] * df_merged['Close']

# 6. Save the resulting DataFrame to a new CSV file
df_merged.to_csv('gold_recovery_full_with_profit.csv', index=False)
print("New CSV file with profit column saved as 'gold_recovery_full_with_profit.csv'")



New CSV file with profit column saved as 'gold_recovery_full_with_profit.csv'


In [4]:
print("Price dataset columns:", df_price.columns)


Price dataset columns: Index(['date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %'], dtype='object')


In [12]:
import pandas as pd

# Load each dataset (the same way you did before)
df_gold = pd.read_csv('../Dataset/gold_recovery_full.csv', parse_dates=['date'])
df_price = pd.read_csv('../Dataset/Refined Gold Futures Historical Data.csv', parse_dates=['Date'])

# Rename columns for consistency
df_price.rename(columns={'Date': 'date', 'Price': 'Close'}, inplace=True)

# Convert both to date (dropping the time part)
df_gold['date'] = df_gold['date'].dt.date
df_price['date'] = df_price['date'].apply(lambda x: x.date())

print("Min/Max in gold data:", df_gold['date'].min(), df_gold['date'].max())
print("Min/Max in price data:", df_price['date'].min(), df_price['date'].max())

print("Unique days in gold data:", df_gold['date'].nunique())
print("Unique days in price data:", df_price['date'].nunique())

common_dates = set(df_gold['date']).intersection(set(df_price['date']))
print("Number of overlapping days:", len(common_dates))


Min/Max in gold data: 2016-01-15 2018-08-18
Min/Max in price data: 2016-01-04 2018-12-31
Unique days in gold data: 947
Unique days in price data: 755
Number of overlapping days: 656
